<a href="https://colab.research.google.com/github/virnarula/card-counting-ai/blob/master/Full%20Repo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Game Model


In [1]:
import enum
import random

## Suit

In [2]:
class Suit(enum.Enum):
    CLUBS = 1
    SPADES = 2
    DIAMONDS = 3
    HEARTS = 4

## Value

In [3]:
class Value(enum.IntEnum):
    ACE = 1
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10
    JACK = 11
    QUEEN = 12
    KING = 13

## Card

In [4]:
class Card:
    def __init__(self, suit, value):
        self.suit = suit
        self.value = value

    def getSuit(self):
        return self.suit

    def getValue(self):
        return self.value

    def getName(self):
        return str(self.value) + " of " + str(self.suit)

    def __eq__(self, other):
        if self.suit == other.suit and self.value == other.value:
            return True
        return False

    def __str__(self):
        return str(int(self.value)) + " of " + str(self.suit)


## Deck

In [5]:
class Deck:
    def __init__(self, decks):
        self.deck = []
        self.used_deck = []
        self.decks = decks
        for i in range(0, decks):
            for suit in Suit:
                for value in Value:
                    self.deck.append(Card(suit, value))

    def draw(self):
        to_return = self.deck.pop()
        self.used_deck.append(to_return)
        if len(self.deck) < 8:
            self.__init__(self.decks)
        return to_return

    def size(self):
        return len(self.deck)

    def swap(self, i, j):
        if i > self.size() - 1 or i < 0 or j > self.size() - 1 or j < 0:
            raise Exception("value was out of range")

        temp = self.deck[i]
        self.deck[i] = self.deck[j]
        self.deck[j] = temp

    def shuffle(self):
        j = 0
        deck_length = len(self.deck)
        for i in range(0, deck_length):
            j = random.randint(i, deck_length - 1)
            self.swap(i, j)

    def printDeck(self):
        for card in self.deck:
            print(card.getName())


## Game

In [6]:
class Game:
    def __init__(self, decks, money):
        self.decks = decks
        self.deck = Deck(decks)
        self.deck.shuffle()
        self.money = money
        self.starting_money = money
        self.bet = 0
        self.player_cards = []
        self.dealer_cards = []
        self.rounds_played = 0
        self.wins = 0
        self.ties = 0
        self.losses = 0
    
    def make_bet(self, bet):
        if bet <= 0 or bet > self.money:
            raise Exception("Bet is not in acceptable range. Bet: " + str(bet))
        self.bet = bet
    
    def deal_cards(self):
        assert not self.player_cards
        assert not self.dealer_cards
        assert self.bet > 0
        self.dealer_cards.append(self.deck.draw())
        self.dealer_cards.append(self.deck.draw())
        self.player_cards.append(self.deck.draw())
        self.player_cards.append(self.deck.draw())

    def hit(self):
        value = Game.evaluateHand(self.player_cards)
        if value == -1 or value >= 21:
            raise Exception("Cannot hit. Current hand value: " + str(value))
        self.player_cards.append(self.deck.draw())
    
    def dealer_hit(self):
        value = Game.evaluateHand(self.dealer_cards)
        while value < 16 and value != -1:
            self.dealer_cards.append(self.deck.draw())
            value = Game.evaluateHand(self.dealer_cards)

    def end_round(self):
       players_hand = Game.evaluateHand(self.player_cards)
       dealers_hand = Game.evaluateHand(self.dealer_cards)

       if players_hand == dealers_hand:
           self.clear_round()
           self.ties += 1
       elif players_hand > dealers_hand:
            self.money += self.bet
            self.wins += 1
            self.clear_round()
       else:
            self.money -= self.bet
            self.losses += 1
            self.clear_round()

    def clear_round(self):
        self.dealer_cards.clear()
        self.player_cards.clear()
        self.bet = 0
        self.rounds_played += 1

    def isBlackjack(hand):
        if len(hand) != 2:
            return False
        
        if int(hand[0].value) > 10 and int(hand[1].value) == 1:
            return True
        
        hand.reverse()

        if int(hand[0].value) > 10 and int(hand[1].value) == 1:
            return True

        return False

    def evaluateHand(hand):
        if Game.isBlackjack(hand):
            return 22

        sum = 0
        aces = 0

        for card in hand:
            if card.getValue().value > 10:
                sum += 10
            elif card.getValue() == Value.ACE:
                sum += 1
                aces += 1
            else:
                sum += card.getValue().value

        while aces > 0 and sum <= 11:
            sum += 10
            aces -= 1

        if sum > 21:
            return -1

        return sum

# User Interface

In [7]:
class user_interface:
    def getBet(game):
        print("You have " + str(game.money) + " dollars")
        bet = int(input("How much would you like to bet? "))
        return bet

    def print_deal(game):
        print("The dealer shows a " + str(game.dealer_cards[0]))
        print("You have a " + str(game.player_cards[0]))
        print("You have a " + str(game.player_cards[1]))

    def get_move(game):
        move = int(input("Enter 0 to stay, 1 to hit: "))
        if move == 1:
            game.hit()
            print_hit(game)
        elif move == 0:
            game.dealer_hit()
            print_dealer_hit(game)
        else:
            get_move(game)

    def print_hit(game):
        card = game.player_cards[len(game.player_cards) - 1]
        print("You got a " + str(card))
        print()
        get_move(game)

    def print_dealer_hit(game):
        print()
        print("The dealer shows: ")
        for card in game.dealer_cards:
            print(str(card.value) + " of " + str(card.suit))
        print("Dealers hand is worth " +  str(model.Game.evaluateHand(game.dealer_cards)))
        print()
        print("You have: ")
        for card in game.player_cards:
            print(str(card.value) + " of " + str(card.suit))
        print("Your hand is worth " + str(model.Game.evaluateHand(game.player_cards)))
        game.end_round()
        print()
        
    def play_game():
        game = model.Game(1, 500)
        while game.money > 0:
            game.make_bet(getBet(game))
            game.deal_cards()
            print_deal(game)
            get_move(game)
        print()
        print("You ran out of money :(")


# Create Keras Model


In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(55,)))
    model.add(tf.keras.layers.Dense(100, activation="relu"))
    model.add(tf.keras.layers.Dense(25, activation="relu"))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    return model

def compile_model(model):
    model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
    return model

def create_betting_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(52,)))
    model.add(tf.keras.layers.Dense(100, activation="relu"))
    model.add(tf.keras.layers.Dense(25, activation="relu"))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    return model

def compile_betting_model(model):
    model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
    return model


# Preprocessor

In [9]:
class Preprocessor:
    def used_card(game):
        cards = game.deck.used_deck
        to_return = []
        for i in range(52):
            to_return.append(0)

        for card in cards:
            index = int(card.value) - 1
            if card.suit == Suit.HEARTS:
                fact = 0
            elif card.suit == Suit.SPADES:
                fact = 1
            elif card.suit == Suit.DIAMONDS:
                fact = 2
            else:
                fact = 3
            index += fact * 13
            to_return[index] = 1

        return to_return

    def get_player_hand_value(game):
        return Game.evaluateHand(game.player_cards)
    
    def get_player_aces(game):
        cards = game.player_cards
        to_return = 0
        for card in cards:
            if card.value == Value.ACE:
                to_return += 1
        return to_return
    
    def get_dealer_card(game):
        return int(game.dealer_cards[0].value)

    def get_final_data(game):
        training = []
        training.append(Preprocessor.get_player_hand_value(game))
        training.append(Preprocessor.get_player_aces(game))
        training.append(Preprocessor.get_dealer_card(game))
        return np.asarray(training + Preprocessor.used_card(game))


# Trainer

In [10]:
TRAINING_ROUNDS = 1000000

class Trainer:
    def __init__(self, model):
        self.game = Game(1, TRAINING_ROUNDS)
        self.log = ""
        self.rounds = 0
        self.model = model
        self.training_rounds = TRAINING_ROUNDS
        self.input= []
        self.rounds_played = 0
        self.hit = False
        self.won = False
        self.train_x = []
        self.train_y = []

    def train(self):
        for i in range(self.training_rounds):
            self.get_move()
            self.get_outcome()
            # self.learn_from_move()
            self.game.end_round()
            self.rounds_played+=1
            if self.rounds_played % 100000 == 0:
                print(self.rounds_played)
        self.learn()
            
    def get_move(self):
        self.game.make_bet(1)
        self.game.deal_cards()
        self.input = Preprocessor.get_final_data(self.game)
        if self.rounds_played % 2 == 0:
            output = 1
        else:
            output = 0
        # output = self.model.predict(self.input)[0]

        if Game.evaluateHand(self.game.player_cards) >= 21 or output == 0:
            self.hit = False
        else:
            self.game.hit()
            self.hit = True
        
        self.game.dealer_hit()

    def get_outcome(self):
        if Game.evaluateHand(self.game.player_cards) >= Game.evaluateHand(self.game.dealer_cards):
            self.won = True
        else:
            self.won = False

        # print(self.won)
        self.train_x.append(np.asarray(self.input))
        if self.won:
            self.train_y.append(np.array([1]))
        else:
            self.train_y.append(np.array([0]))

    def learn(self):
        print("Learning")
        self.model.fit(np.asarray(self.train_x), np.asarray(self.train_y), batch_size=25, epochs=5)


# Bet Trainer


In [11]:
BET_TRAINING_ROUNDS = 100000

class BetTrainer:
    def __init__(self, betting_model, hitting_model):
        self.betting_model = betting_model
        self.hitting_model = hitting_model
        self.train_x = []
        self.train_y = []
        self.training_rounds = BET_TRAINING_ROUNDS
        self.game = Game(1, self.training_rounds)
    
    def train(self):
        for i in range(self.training_rounds):
            if i % 10000 == 0:
                print(i)
            self.input = Preprocessor.used_card(self.game)
            self.game.make_bet(1)
            self.game.deal_cards()
            self.play_move()
            self.game.dealer_hit()
            self.train_x.append(self.input)
            if Game.evaluateHand(self.game.player_cards) >= Game.evaluateHand(self.game.dealer_cards):
                self.train_y.append(np.array([1]))
            else:
                self.train_y.append(np.array([0]))
            self.game.end_round()
        self.learn()

    def play_move(self):
        input = []
        input.append(Preprocessor.get_final_data(self.game))
        output = self.hitting_model.predict(np.asarray(input))
        if output[0][0] <= 0.5 or Game.evaluateHand(self.game.player_cards) >= 21:
            return
        else:
            self.game.hit()
    
    def learn(self):
        self.betting_model.fit(np.asarray(self.train_x), np.asarray(self.train_y), batch_size=100, epochs=5)


# Tester


In [12]:
class Tester:
    def __init__(self, model, betting_model, money, decks):
        self.model = model
        self.betting_model = betting_model
        self.starting_money = money
        self.money = money
        self.game = Game(decks, money)


    def play_rounds(self, rounds):
        for i in range(rounds):
            self.game.make_bet(self.make_bet())
            self.game.deal_cards()
            if self.play_move(Preprocessor.get_final_data(self.game)) == 1:
                self.game.hit()
            self.game.dealer_hit()
            self.game.end_round()

    def make_bet(self):
        input = []
        input.append(Preprocessor.used_card(self.game))
        output = self.betting_model.predict(np.asarray(input))[0][0]
        if output <= 0:
            return 1
        if output > 1:
            return self.game.money
        output *= self.game.money
        if output < 1:
            return 1
        return int(output / 1000)

    def play_move(self, input):
        input = []
        input.append(Preprocessor.get_final_data(self.game))
        output = self.model.predict(np.asarray(input))
        if output[0][0] <= 0.5 or Game.evaluateHand(self.game.player_cards) >= 21:
            return 0
        return 1

    def summary(self):
        to_return = "Started with $" + str(self.starting_money) + "\n"
        to_return += "Current money: $" + str(self.game.money) + "\n"
        to_return += "Played " + str(self.game.rounds_played) + " rounds\n"
        to_return += "Wins: " + str(self.game.wins) + " Ties: " + str(self.game.ties) + " Losses: " + str(self.game.losses) + "\n"
        return to_return


# Training Model

In [ ]:
from os import path

if path.exists("/blackjack/blackjack.model"):
    print("Found saved model")
    ai = tf.keras.models.load_model("/blackjack/blackjack.model")
else:
    print("Could not find saved model. Creating and training new.")
    trainer = Trainer(compile_model(create_model()))
    trainer.train()
    ai = trainer.model
    ai.save("/blackjack/hitting.model")
    print("Saved model to /blackjack/hitting.model")

Could not find saved model. Creating and training new.
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
Learning
Epoch 1/5
 9143/40000 [=====>........................] - ETA: 30s - loss: 0.1303 - accuracy: 0.9389

In [ ]:
if path.exists("/blackjack/betting.model"):
    print("found saved betting model")
    betting_model = tf.keras.models.load_model("/blackjack/betting.model")
else:
    bet_trainer = BetTrainer(compile_betting_model(create_betting_model()), ai)
    bet_trainer.train()
    betting_model = bet_trainer.betting_model
    betting_model.save("/blackjack/betting.model")
    print("Saving model to /blackjack/betting.model")

# Testing Model


In [ ]:
tester = Tester(ai, betting_model, 10000000, 1)
tester.play_rounds(10000)
print(tester.summary())